In [ ]:
#!pip install langchain==0.0.339 openai typing pydantic=="2.5.*" replicate==0.20.* SQLAlchemy==2.0.23 google-generativeai==0.2.* --upgrade


## About

In this notebook you will see how zero-shot and one-shot learning is used to assess relevancy of a product to a customer query

In [3]:
import os
from langchain.chat_models import ChatOpenAI, ChatGooglePalm
from langchain.prompts import ChatPromptTemplate
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.llms import Replicate

from langchain.schema import StrOutputParser
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache
from langchain.callbacks import get_openai_callback

from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain.output_parsers import CommaSeparatedListOutputParser

import pandas as pd
import json
from ipywidgets import interact
import ipyplot
import code.helper
from dotenv import load_dotenv


In [2]:
set_llm_cache(SQLiteCache(database_path=".langchain.db"))


In [4]:
load_dotenv("../.env") 

True

In [5]:
llm_openai = ChatOpenAI(model="gpt-4")


## Prompts
Zero shot prompt describes the task and provides query and item description

One shot prompt provides example query-item description pairs besides the task and inputs

In [18]:
zero_shot_template = """
You are an helpful e-commerce assistant.
You will judge relevancy of an item to a query and respond one of the three options below.
1. 'relevant' if the item is relevant to the query
2. 'partially relevant' if the item is somewhat relevant to the query
3. 'irrelevant' if the item is not relevant to the query

query: {query}
item: {item_description}

Let's think step by step and explain your reasoning.
Answer
"""

In [16]:
one_shot_template = """
You are an helpful e-commerce assistant.
You will judge relevancy of an item to a query and respond one of the three options below.
1. 'relevant' if the item is relevant to the query
2. 'partially relevant' if the item is somewhat relevant to the query
3. 'irrelevant' if the item is not relevant to the query

Here is an example of the task.
one shot query: 30 oz tumbler with handle
one shot item: Simple Modern 30 oz Tumbler with Handle and Straw Lid | Insulated Cup Reusable Stainless Steel Water Bottle Travel Cupholder Friendly | Gifts for Women Him Her | Trek Collection | 80s Mix
Answer: relevant

query: {query}
item: {item_description}

Let's think step by step and explain your reasoning.
Answer: """

In [17]:
prompt = PromptTemplate(template=zero_shot_template, input_variables=["query", "item_description"])

In [7]:
chain_gpt4 = LLMChain(llm=llm_openai, prompt=prompt, output_parser=StrOutputParser())


In [8]:
def get_relevancy(query, item):
    with get_openai_callback() as cb:
        result = chain_gpt4.run(query=query,item_description=item)
        print (result )

In [9]:
query = "adidas mens shoes 15"
item = "adidas Originals Men's Superstar Foundation Shoes Sneaker, White/White/White, 15"
get_relevancy(query, item)

'relevant'

The item is a men's shoe by Adidas, which matches the brand and gender specified in the query. It is also a size 15, which is what the query is looking for. Therefore, the item is relevant to the query.


In [10]:
query = "adidas mens shoes 15"
item = "adidas Ultraboost 19 Shoes Men's, White, Size 9"
get_relevancy(query, item)

'partially relevant'

The item is a men's shoe from Adidas which matches with the query. However, the size specified in the query is 15 but the item is of size 9. Hence, it's partially relevant since it fulfills two out of three criteria from the query.


In [11]:
query = "adidas mens shoes 15"
item = "Merrell Hiking Boots Womens', White, Size 9"
get_relevancy(query, item)

'irrelevant'

The query is requesting for adidas mens shoes in the size 15. The item listed is a women's Merrell hiking boot in the size 9. The brand, gender, shoe type, and size all do not match the search query, making it irrelevant to the user's needs.


#### Let's try a harder example

In [12]:
query = "gifts for men"
item = "Merrell Hiking Boots Womens', White, Size 9"
get_relevancy(query, item)

'irrelevant'

Reasoning:
The query is looking for gifts for men, but the item in question is a pair of women's hiking boots. Therefore, it is not relevant to the query.


In [13]:
query = "gifts for men"
item = "DualShock 4 Wireless Controller for PlayStation 4 - Glacier White"
get_relevancy(query, item)

'relevant'

The item, a DualShock 4 Wireless Controller for PlayStation 4, can be considered as a relevant gift for men. PlayStation is a popular gaming console, and the controller is an integral part of the gaming experience. While it's not specific to men, it's not unusual for men to have an interest in video games. Therefore, this item can be seen as a potential gift for men.


In [14]:
query= "gifts for men"
item="""Brandzig U-40 Pet Insulin Syringes 29G 3/10cc, 1/2" 100-Pack"""
get_relevancy(query, item)

'irrelevant'

Reasoning:
The query is for 'gifts for men'. The item is a pack of insulin syringes, which is a medical supply, not typically given as a gift. Furthermore, the item does not specify anything to suggest it is particularly suitable or designed for men. Therefore, it is not relevant to the query.
